In [36]:
import pandas as pd
import numpy as np

### 1. Open Air Quality Dataset

In [9]:
df_airQuality = pd.read_csv('../airQuality_2019/datos201912.csv', delimiter = ";")

In [69]:
df_airQuality.head(3)

,PROVINCIA,MUNICIPIO,ESTACION,MAGNITUD,PUNTO_MUESTREO,ANO,MES,D01,V01,D02,...,D27,V27,D28,V28,D29,V29,D30,V30,D31,V31
0,28,79,4,1,28079004_1_38,2019,1,18.0,V,20.0,...,15.0,V,15.0,V,15.0,V,14.0,V,14.0,V
1,28,79,4,1,28079004_1_38,2019,2,13.0,V,13.0,...,19.0,V,20.0,V,0.0,N,0.0,N,0.0,N
2,28,79,4,1,28079004_1_38,2019,3,18.0,V,18.0,...,6.0,V,7.0,V,6.0,V,4.0,V,4.0,V


In [11]:
df_airQuality.dtypes

PROVINCIA           int64
MUNICIPIO           int64
ESTACION            int64
MAGNITUD            int64
PUNTO_MUESTREO     object
                   ...   
V29                object
D30               float64
V30                object
D31               float64
V31                object
Length: 69, dtype: object

### 2. Open Monitoring Stations Dataset

In [87]:
df_stations = pd.read_excel('../monitoring_stations/stations.xls', index_col=0)
# extract columns needed 
df_stations_locations = df_stations[['CODIGO_CORTO', 'LONGITUD', 'LATITUD']]
# df_stations_locations = df_stations_locations.drop(columns = [1])
stations_locations = df_stations_locations.loc[: , "CODIGO_CORTO":"LATITUD"]
stations_locations.head()

,CODIGO_CORTO,LONGITUD,LATITUD
CODIGO,,,
28079004,4,-3.712257,40.423882
28079008,8,-3.682316,40.421553
28079011,11,-3.677349,40.451473
28079016,16,-3.639242,40.440046
28079017,17,-3.713317,40.347147


### 3. Extract Carbon Monoxide

In [8]:
# extract magnitude 6 which corresponds to carbon monoxide (CO)
magnitude_6 = df_airQuality.loc[df_airQuality['MAGNITUD'] == 6]

In [70]:
# replace all zeros with nan (this is useful to calculate the mean later)
magnitude_6 = magnitude_6.replace(0, np.NaN)
magnitude_6.head(3)

,PROVINCIA,MUNICIPIO,ESTACION,MAGNITUD,PUNTO_MUESTREO,ANO,MES,D01,V01,D02,...,D27,V27,D28,V28,D29,V29,D30,V30,D31,V31
12,28,79,4,6,28079004_6_48,2019,1,0.8,V,1.0,...,0.4,V,0.3,V,0.4,V,0.4,V,0.3,V
13,28,79,4,6,28079004_6_48,2019,2,0.3,V,0.4,...,0.7,V,0.8,V,NaN,N,NaN,N,NaN,N
14,28,79,4,6,28079004_6_48,2019,3,0.6,V,0.6,...,0.5,V,0.6,V,0.5,V,0.5,V,0.4,V


In [38]:
# remove validation columns (with V as first letter)
magnitude_6_noVColumns = magnitude_6.loc[:, ~magnitude_6.columns.str.startswith('V')]

In [39]:
# drop columns not needed 
magnitude_6_reduced = magnitude_6_noVColumns.drop(columns=['PROVINCIA', 'MUNICIPIO','PUNTO_MUESTREO','MES'])

In [71]:
magnitude_6_reduced.head(3)

,ESTACION,MAGNITUD,ANO,D01,D02,D03,D04,D05,D06,D07,...,D23,D24,D25,D26,D27,D28,D29,D30,D31,carbon_monoxid_mean
12,4,6,2019,0.8,1.0,0.9,1.0,0.9,1.0,1.0,...,0.3,0.5,0.6,0.5,0.4,0.3,0.4,0.4,0.3,0.687097
13,4,6,2019,0.3,0.4,0.4,0.7,0.8,0.8,0.8,...,0.8,0.7,0.8,0.7,0.7,0.8,NaN,NaN,NaN,0.653571
14,4,6,2019,0.6,0.6,0.5,0.3,0.4,0.3,0.4,...,0.5,0.5,0.4,0.3,0.5,0.6,0.5,0.5,0.4,0.470968


In [41]:
# extract only days from the dataframe
days = magnitude_6_reduced.loc[: , "D01":"D31"]
# calculate new column with mean aggregating all columns extracted above
magnitude_6_reduced['carbon_monoxid_mean'] = days.mean(axis=1)

In [72]:
magnitude_6_reduced.head(3)

,ESTACION,MAGNITUD,ANO,D01,D02,D03,D04,D05,D06,D07,...,D23,D24,D25,D26,D27,D28,D29,D30,D31,carbon_monoxid_mean
12,4,6,2019,0.8,1.0,0.9,1.0,0.9,1.0,1.0,...,0.3,0.5,0.6,0.5,0.4,0.3,0.4,0.4,0.3,0.687097
13,4,6,2019,0.3,0.4,0.4,0.7,0.8,0.8,0.8,...,0.8,0.7,0.8,0.7,0.7,0.8,NaN,NaN,NaN,0.653571
14,4,6,2019,0.6,0.6,0.5,0.3,0.4,0.3,0.4,...,0.5,0.5,0.4,0.3,0.5,0.6,0.5,0.5,0.4,0.470968


In [48]:
# drop all columns with day values (days)
magnitude_6_dropColumns = magnitude_6_reduced.drop(columns = days)

In [73]:
magnitude_6_dropColumns.head(3)

,ESTACION,MAGNITUD,ANO,carbon_monoxid_mean
12,4,6,2019,0.687097
13,4,6,2019,0.653571
14,4,6,2019,0.470968


In [150]:
carbon_monoxide_2019 = magnitude_6_dropColumns.groupby('ESTACION') \
                .agg({'carbon_monoxid_mean':'mean'}) \
                .rename(columns={0:'monitoring_station','carbon_monoxid_mean':'carbon_monoxide_mean2019'}) 

In [95]:
carbon_monoxide_2019 = carbon_monoxide_2019.rename(columns={'ESTACION':'monitoring_station'})

In [147]:
carbon_monoxide_2019.head(15)

,monitoring_station,carbon_monoxide_mean2019
0,4,0.417117
1,8,0.274271
2,16,0.295085
3,18,0.468886
4,24,0.207405
5,35,0.451720
6,36,0.304368
7,39,0.331003
8,56,0.326461
9,57,0.277555


In [116]:
co_mean_2019 = carbon_monoxide_2019.merge(stations_locations, left_on='monitoring_station', right_on='CODIGO_CORTO')
co_mean_2019 = co_mean_2019.drop(columns = ['CODIGO_CORTO'])
co_mean_2019 = co_mean_2019.rename(columns={'monitoring_station': 'station','carbon_monoxide_mean2019': 'co','LONGITUD':'longitude', 'LATITUD':'latitude'})
co_mean_2019.head()

,station,co,longitude,latitude
0,4,0.417117,-3.712257,40.423882
1,8,0.274271,-3.682316,40.421553
2,16,0.295085,-3.639242,40.440046
3,18,0.468886,-3.731836,40.394782
4,24,0.207405,-3.747345,40.419358


In [117]:
co_mean_2019.to_csv('../airQuality_2019/codata.csv', index=False)

### 4. Extract Particulate Matter 10 

In [125]:
# extract magnitude 6 which corresponds to carbon monoxide (CO)
magnitude_8 = df_airQuality.loc[df_airQuality['MAGNITUD'] == 8]
# replace all zeros with nan (this is useful to calculate the mean later)
magnitude_8 = magnitude_8.replace(0, np.NaN)
magnitude_8.head(3)

,PROVINCIA,MUNICIPIO,ESTACION,MAGNITUD,PUNTO_MUESTREO,ANO,MES,D01,V01,D02,...,D27,V27,D28,V28,D29,V29,D30,V30,D31,V31
36,28,79,4,8,28079004_8_8,2019,1,71.0,V,84.0,...,32.0,V,25.0,V,30.0,V,29.0,V,18.0,V
37,28,79,4,8,28079004_8_8,2019,2,18.0,V,26.0,...,78.0,V,89.0,V,NaN,N,NaN,N,NaN,N
38,28,79,4,8,28079004_8_8,2019,3,64.0,V,58.0,...,57.0,V,62.0,V,55.0,V,44.0,V,42.0,V


In [126]:
# remove validation columns (with V as first letter)
magnitude_8_noVColumns = magnitude_8.loc[:, ~magnitude_8.columns.str.startswith('V')]
# drop columns not needed 
magnitude_8_reduced = magnitude_8_noVColumns.drop(columns=['PROVINCIA', 'MUNICIPIO','PUNTO_MUESTREO','MES'])
magnitude_8_reduced.head(3)

,ESTACION,MAGNITUD,ANO,D01,D02,D03,D04,D05,D06,D07,...,D22,D23,D24,D25,D26,D27,D28,D29,D30,D31
36,4,8,2019,71.0,84.0,77.0,86.0,75.0,82.0,87.0,...,54.0,26.0,51.0,60.0,52.0,32.0,25.0,30.0,29.0,18.0
37,4,8,2019,18.0,26.0,41.0,74.0,74.0,66.0,73.0,...,78.0,80.0,63.0,78.0,70.0,78.0,89.0,NaN,NaN,NaN
38,4,8,2019,64.0,58.0,40.0,18.0,33.0,23.0,29.0,...,61.0,47.0,45.0,44.0,24.0,57.0,62.0,55.0,44.0,42.0


In [127]:
# extract only days from the dataframe
days = magnitude_8_reduced.loc[: , "D01":"D31"]
# calculate new column with mean aggregating all columns extracted above
magnitude_8_reduced['particulate_matter'] = days.mean(axis=1)
magnitude_8_reduced.head(3)

,ESTACION,MAGNITUD,ANO,D01,D02,D03,D04,D05,D06,D07,...,D23,D24,D25,D26,D27,D28,D29,D30,D31,particulate_matter
36,4,8,2019,71.0,84.0,77.0,86.0,75.0,82.0,87.0,...,26.0,51.0,60.0,52.0,32.0,25.0,30.0,29.0,18.0,61.129032
37,4,8,2019,18.0,26.0,41.0,74.0,74.0,66.0,73.0,...,80.0,63.0,78.0,70.0,78.0,89.0,NaN,NaN,NaN,62.000000
38,4,8,2019,64.0,58.0,40.0,18.0,33.0,23.0,29.0,...,47.0,45.0,44.0,24.0,57.0,62.0,55.0,44.0,42.0,44.677419


In [146]:
# drop all columns with day values (days)
magnitude_8_dropColumns = magnitude_8_reduced.drop(columns = days)
magnitude_8_dropColumns.head(3)

,ESTACION,MAGNITUD,ANO,particulate_matter
36,4,8,2019,61.129032
37,4,8,2019,62.000000
38,4,8,2019,44.677419


In [154]:
particulate_matter_2019 = magnitude_8_dropColumns.groupby('ESTACION') \
                .agg({'particulate_matter':'mean'}) \
                .rename(columns={'particulate_matter':'particulate_matter_mean2019'}) \
                .reset_index()
              
particulate_matter_2019 = particulate_matter_2019.rename(columns={'ESTACION':'monitoring_station'})
particulate_matter_2019

,monitoring_station,particulate_matter_mean2019
0,4,39.758701
1,8,51.106368
2,11,39.355594
3,16,34.406036
4,17,39.385650
5,18,32.360298
6,24,20.565214
7,27,35.649309
8,35,36.129512
9,36,36.107444


In [155]:
pm_mean_2019 = particulate_matter_2019.merge(stations_locations, left_on='monitoring_station', right_on='CODIGO_CORTO')
pm_mean_2019 = pm_mean_2019.drop(columns = ['CODIGO_CORTO'])
pm_mean_2019 = pm_mean_2019.rename(columns={'monitoring_station': 'station','particulate_matter_mean2019': 'pm','LONGITUD':'longitude', 'LATITUD':'latitude'})
pm_mean_2019.head()

,station,pm,longitude,latitude
0,4,39.758701,-3.712257,40.423882
1,8,51.106368,-3.682316,40.421553
2,11,39.355594,-3.677349,40.451473
3,16,34.406036,-3.639242,40.440046
4,17,39.385650,-3.713317,40.347147


In [156]:
pm_mean_2019.to_csv('../airQuality_2019/pmdata.csv', index=False)